In [1]:
import pandas as pd
import numpy as np
import os
import json
import cv2
import random
import sklearn
import pydot
import tqdm
from datetime import datetime

from matplotlib import image as mpimg
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import yaml
import torch
from pathlib import Path

import ultralytics
from ultralytics import YOLO
from albumentations import (
    HorizontalFlip,
    VerticalFlip,
    RandomRotate90,
    RandomBrightnessContrast,
    RandomGamma,
    Compose,
    ImageOnlyTransform,
    Resize,
)

In [4]:
data_dir = 'D:/Benji/dataset/'
train_images_dir = os.path.join(data_dir, 'images', 'train')
train_labels_dir = os.path.join(data_dir, 'labels', 'train')
augmented_images_dir = 'D:/Benji/dataset_aug/images/train'
augmented_labels_dir = 'D:/Benji/dataset_aug/labels/train'

os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_labels_dir, exist_ok=True)

transform = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    RandomRotate90(p=0.5),
    RandomBrightnessContrast(p=0.2),
    RandomGamma(p=0.2),
    ImageOnlyTransform(Resize(416, 416)),
])

image_files = [f for f in os.listdir(train_images_dir) if f.endswith('.jpg')]

for image_file in image_files:
    image_path = os.path.join(train_images_dir, image_file)
    image = cv2.imread(image_path)
    label_file = os.path.splitext(image_file)[0] + '.txt'
    label_path = os.path.join(train_labels_dir, label_file)
    
    augmented = transform(image=image)
    augmented_image = augmented['image']
    augmented_image_path = os.path.join(augmented_images_dir, image_file)
    cv2.imwrite(augmented_image_path, augmented_image)

    augmented_label_path = os.path.join(augmented_labels_dir, label_file)
    with open(label_path, 'r') as label_file:
        augmented_label = label_file.read()
    with open(augmented_label_path, 'w') as label_file:
        label_file.write(augmented_label)

NotImplementedError: 